<div style="padding: 20px; background-color: #f4ecf7; border-radius: 10px; border: 1px solid #8e44ad;">
    <h1 style="color: #5b2c6f; font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;">🧠 Step 03: The Intelligence</h1>
    <h2 style="color: #8e44ad;">Graph Analytics and Structural Hole Detection</h2>
    <p style="font-size: 1.1em; color: #2c3e50;">We have the grid and the data. Now, we turn this into an intelligent network. We aren't just looking for where people live; we are looking for <strong>Structural Holes</strong> in the business ecosystem.</p>
    <hr>
    <p><strong>Goal:</strong> Transform hexagons into a spatial graph, calculate synergy scores, and visualize the optimal sites for our coffee shop in 3D.</p>
</div>

<div style="margin-top: 30px;">
    <h2 style="color: #2e86c1; border-bottom: 2px solid #2e86c1; padding-bottom: 10px;">🧠 Concept: What is a Spatial Graph?</h2>
    <p>Imagine the city as a web of connected hubs:</p>
    <ul>
        <li><strong>Nodes:</strong> These are the hexagons from Step 02. Each node "knows" its population, social grade, and which amenities are inside it.</li>
        <li><strong>Edges:</strong> These are the walking paths between hexagons. If two hexagons touch, we draw a line (an edge) between them.</li>
    </ul>
</div>

In [ ]:
import geopandas as gpd
import networkx as nx
import h3
import pandas as pd
import pydeck as pdk

# Load our Master Grid from Step 02
h3_grid = gpd.read_parquet("data/outputs/camden_h3_grid.parquet")

# Load POIs from Step 01
pois = gpd.read_file("data/processed/camden_pois.geojson")

print("Data loaded. Let's build the brain of our model.")

<div style="margin-top: 30px;">
    <h2 style="color: #117a65; border-bottom: 2px solid #117a65; padding-bottom: 10px;">🕸️ Task 1: Building the Graph with NetworkX</h2>
    <p>We connect each hexagon to its 6 immediate neighbors. We weight these edges based on distance using <strong>Inverse Distance Weighting (IDW)</strong>.</p>
    <p><strong>Weighting Formula:</strong></p>
    <p>$$w_{uv} = \frac{1}{dist(u, v) + 1}$$</p>
</div>

In [ ]:
G = nx.Graph()

# 1. Create Nodes from Hexagons
for idx, row in h3_grid.iterrows():
    G.add_node(row['h3_index'], population=row['population'], 
               x=row.geometry.centroid.x, y=row.geometry.centroid.y)

# 2. Create Edges between Adjacently Touching Hexagons
for h3_idx in h3_grid['h3_index']:
    neighbors = h3.k_ring(h3_idx, 1)
    for neighbor in neighbors:
        if neighbor != h3_idx and G.has_node(neighbor):
            # Calculate distance between centers for the weight
            p1 = (G.nodes[h3_idx]['x'], G.nodes[h3_idx]['y'])
            p2 = (G.nodes[neighbor]['x'], G.nodes[neighbor]['y'])
            d = ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
            G.add_edge(h3_idx, neighbor, distance=d, weight=1/(d+1))

print(f"Spatial Graph complete: {G.number_of_nodes()} hex-nodes and {G.number_of_edges()} infrastructure edges.")

<div style="margin-top: 30px;">
    <h2 style="color: #884ea0; border-bottom: 2px solid #884ea0; padding-bottom: 10px;">📊 Task 2: Detecting Structural Holes (The Site Score)</h2>
    <p><strong>Retail Synergy Concept:</strong> A coffee shop thrives where people are active (Synergy Nodes like Gyms) but where competition is low. We call these gaps <strong>Structural Holes</strong>.</p>
    <p><strong>Site Score Formula:</strong></p>
    <p>$$Score_H = (Pop_H \times 0.5) + (\text{Synergy Nodes} \times 5) - (\text{Competitors} \times 20)$$</p>
</div>

In [ ]:
# Mapping POI counts to hexagons for the score
hex_pois = gpd.sjoin(pois, h3_grid.to_crs(pois.crs), how="inner", op="within")

# Helper to count types
def calculate_site_score(h3_id):
    local_pois = hex_pois[hex_pois['h3_index'] == h3_id]
    
    competitors = len(local_pois[local_pois['amenity'] == 'cafe'])
    synergy = len(local_pois[local_pois['amenity'].isin(['gym', 'university', 'office'])])
    pop = G.nodes[h3_id]['population']
    
    # Calculation (High Synergies + Pop, High Penalty for Competitors)
    return (pop * 0.01) + (synergy * 5) - (competitors * 25)

h3_grid['site_score'] = h3_grid['h3_index'].apply(calculate_site_score)

print("Site potential scores calculated across all hexagons.")

<div style="margin-top: 30px;">
    <h2 style="color: #1a5276; border-bottom: 2px solid #1a5276; padding-bottom: 10px;">🌐 Visual Vision: Interactive 3D Mapping</h2>
    <p>We use <strong>Pydeck</strong> to extrude our hexagons. The height represents the <strong>Potential Score</strong>. High, bright towers are our "Gold Mines."</p>
</div>

In [ ]:
# Ensure coordinates are back to WGS84 for Pydeck
viz_df = h3_grid.to_crs(epsg=4326)
viz_df['lat'] = viz_df.geometry.centroid.y
viz_df['lon'] = viz_df.geometry.centroid.x

layer = pdk.Layer(
    "H3HexagonLayer",
    viz_df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=True,
    get_hexagon="h3_index",
    get_fill_color="[255, (1 - site_score/max(site_score+1))*255, 0, 160]",
    get_elevation="site_score * 50",
)

view_state = pdk.ViewState(latitude=51.54, longitude=-0.14, zoom=12, pitch=50)
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=True)
r.to_html("data/outputs/camden_site_potential.html")

print("Visual dashboard saved to data/outputs/camden_site_potential.html!")
print("Congratulations! You've successfully built a production-grade geospatial retail engine.")